Import the required libraries

In [1]:
from urllib.parse import urlencode, urlparse, parse_qsl
import pandas as pd
import requests

Get the API key

In [2]:
api_key = "YOUR_API_KEY"

Create function to generate latlong from address

In [5]:
def extract_lat_long(address_or_postalcode, data_type = "json"):
    endpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    params = {"address": address_or_postalcode, "key": api_key}
    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    r = requests.get(url)
    if r.status_code not in range(200,299):
        return{}
    try:
        latlng = r.json()["results"][0]["geometry"]["location"]
    except:
        pass
    return latlng.get("lat"), latlng.get("lng")

Create parameters to build the requested table

In [17]:
input = "YOUR_KEYWORD" #keyword
radius = 1000 #maximum distance from address
address = "YOUR_ADDRESS"
lat, long = extract_lat_long(address)

Dataset creation

In [18]:
nearby_endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "key": api_key,
    "location": f"{lat},{long}",
    "radius": radius,
    "keyword": input
}
params_encoded = urlencode(params)
places_url = f"{nearby_endpoint}?{params_encoded}"
request = requests.get(places_url).json()

df = pd.DataFrame(columns=["Név", "Cím", "Értékelés"])

for place in request["results"]:
    name = place["name"]
    ertekeles = place["rating"]
    cim = place["vicinity"]
    df1 = pd.DataFrame([[name, cim, ertekeles]], columns=["Név", "Cím", "Értékelés"])
    df = pd.concat([df,df1])
    
df = df.reset_index()
df = df.drop(columns="index")
df.sort_values("Értékelés", ascending=False)

,Név,Cím,Értékelés
4,Fermento,"Budapest, Csalogány u. 26",4.8
6,Porcellino Grasso Ristorante,"Budapest, Ady Endre u. 19",4.6
1,Due Fratelli,"Budapest, Erőd u. 9",4.4
2,IGEN,"Budapest, Margit körút 60",4.4
0,Pizza Me,"Budapest, Margit körút 91",4.3
7,Marxim,"Budapest, Kis Rókus u. 23",4.3
5,IL Treno,"Budapest, Lövőház u. 2-6",4.1
3,Pizza Forte,"Budapest, Széll Kálmán tér M",3.5
